# Tutorial 03 : Support Vector Machines (SVMs)
by Dr Ivan Olier-Caparroso, last updated: 30/01/22

## Introduction
The aim with this tutorial is to practise the implementation of support vector machines (SVMs) in Python.

As in prior lab, let's import several common modules that we would be using throughout the lab. `pandas` for dataframe manipulation, `matplotlib` for plotting, and `numpy` for arrays and linear algebra operations:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl #also import the whole module as we will use other functions not in pyplot.
import matplotlib.pyplot as plt
%matplotlib inline

In this lab, we will also use the `SVC` module from the `sklearn.svm` package to demonstrate the support vector classifier and the SVM:

In [ ]:
from sklearn.svm import SVC

## Support Vector Classifier

The `SVC()` function can be used to fit a support vector classifier when the argument `kernel="linear"` is used. The `c` argument allows us to specify the cost of a violation to the margin. When the `c` argument is **small**, then the margins will be wide and many support vectors will be on the margin or will violate the margin. When the `c` argument is large, then the margins will be narrow and there will be few support vectors on the margin or violating the margin.

We can use the `SVC()` function to fit the support vector classifier for a given value of the `cost` parameter. Here we demonstrate the use of this function on a two-dimensional example so that we can plot the resulting decision boundary. Let's start by generating a set of observations, which belong to two classes:

In [ ]:
# Generating random data: 20 observations of 2 features and divide into two classes.
np.random.seed(5)
X = np.random.randn(20,2)
y = np.repeat([1,-1], 10)

X[y == -1] = X[y == -1]+1

Let's plot the data to see whether the classes are linearly separable:

In [ ]:
plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=mpl.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')

Nope; not linear. Next, we fit the support vector classifier:

In [ ]:
svc = SVC(C=1, kernel='linear')
svc.fit(X, y)

We can now plot the support vector classifier. To this, we can create a python function to draw nice plots of SVMs as follows:

In [ ]:
def plot_svc(svc, X, y, h=0.02, pad=0.25):
    x_min, x_max = X[:, 0].min()-pad, X[:, 0].max()+pad
    y_min, y_max = X[:, 1].min()-pad, X[:, 1].max()+pad
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.2)

    plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=mpl.cm.Paired)
    # Support vectors indicated in plot by vertical lines
    sv = svc.support_vectors_
    plt.scatter(sv[:,0], sv[:,1], c='k', marker='x', s=100, linewidths=1)
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.show()
    print('Number of support vectors: ', svc.support_.size)

Remember that Python functions are created with `def`. Also, that the indentation is informative in Python and you will get an error message if it isn't respected. We call the function `plot_svc`. There are several functions from the `matplotlib.pyplot` module that we use such as `contourf` (for drawing filled countours), `scatter` (for plotting scatter plots), `xlim` and `ylim` (for setting axis ranges), and `xlabel` and `ylabel` (for axis labelling). Also, notice the use of the function `show`, which is needed to display the figure. We also use several `numpy` functions such as `meshgrid` (for coordinate matrices), `arange` (for froming evenly spaced values within a given interval), and `reshape` (for reshaping an array). The `plot_svc()` function takes as first argument a `SVC` object, which we just learnt how to use it. The best way to learn about a new language is by understanding how already implemented code works. In an *IPython* console you can use `help()` to access the reference documentation. Sometimes is easier just to google the function.

Let's call the `plot_svc()` function to plot the support vector classifier:

In [ ]:
plot_svc(svc, X, y)

The region of feature space that will be assigned to the −1 class is shown in light blue, and the region that will be assigned to the +1 class is shown in brown. The decision boundary between the two classes is linear (because we used the argument `kernel="linear"`).

The support vectors are plotted with crosses and the remaining observations are plotted as circles; we see here that there
are 13 support vectors. We can determine their identities as follows:

In [ ]:
svc.support_

What if we instead used a smaller value of the `cost` parameter?

In [ ]:
svc2 = SVC(C=0.1, kernel='linear')
svc2.fit(X, y)
plot_svc(svc2, X, y)

Now that a smaller value of the `c` parameter is being used, we obtain a larger number of support vectors, because the margin is now **wider**.

The `sklearn.grid_search` module includes a function `GridSearchCV()` to perform cross-validation. In order to use this function, we pass in relevant information about the set of models that are under consideration. The following command indicates that we want perform 10-fold cross-validation to compare SVMs with a linear kernel, using a range of values of the cost parameter:

In [ ]:
from sklearn.model_selection import GridSearchCV

# Select the optimal C parameter by cross-validation
tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1, 5, 10, 100]}]
clf = GridSearchCV(SVC(kernel='linear'), tuned_parameters, cv=10, scoring='accuracy')
clf.fit(X, y)

We can easily access the cross-validation errors for each of these models:

In [ ]:
clf.cv_results_

The `GridSearchCV()` function stores the best parameters obtained, which can be accessed as follows:

In [ ]:
clf.best_params_

c=0.001 is best according to `GridSearchCV()`. 

As usual, the `predict()` function can be used to predict the class label on a set of test observations, at any given value of the cost parameter. Let's generate a test data set:

In [ ]:
np.random.seed(1)
X_test = np.random.randn(20,2)
y_test = np.random.choice([-1,1], 20)
X_test[y_test == 1] = X_test[y_test == 1]-1

Now we predict the class labels of these test observations. Here we use the
best model obtained through cross-validation in order to make predictions:

In [ ]:
from sklearn.metrics import confusion_matrix
svc2 = SVC(C=0.001, kernel='linear')
svc2.fit(X, y)
y_pred = svc2.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred), index=svc2.classes_, columns=svc2.classes_)

With this value of ${\tt c}$, 14 of the test observations are correctly
classified.

Now consider a situation in which the two classes are linearly separable.
Then we can find a separating hyperplane using the `svm()` function. First we'll give our simulated data a little nudge so that they are linearly separable:

In [ ]:
X_test[y_test == 1] = X_test[y_test == 1] -1
plt.scatter(X_test[:,0], X_test[:,1], s=70, c=y_test, cmap=mpl.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')

Now the observations are **just barely linearly** separable. We fit the support
vector classifier and plot the resulting hyperplane, using a very large value
of `cost` so that no observations are misclassified.

In [ ]:
svc3 = SVC(C=1e5, kernel='linear')
svc3.fit(X_test, y_test)
plot_svc(svc3, X_test, y_test)

No training errors were made and only three support vectors were used.
However, we can see from the figure that the margin is very narrow (because
the observations that are **not** support vectors, indicated as circles, are very close to the decision boundary). It seems likely that this model will perform poorly on test data. Let's try a smaller value of `cost`:

In [ ]:
svc4 = SVC(C=1, kernel='linear')
svc4.fit(X_test, y_test)
plot_svc(svc4, X_test, y_test)

Using `cost=1`, we misclassify a training observation, but we also obtain a much wider margin and make use of five support vectors. It seems likely that this model will perform better on test data than the model with `cost=1e5`.

## Support Vector Machine

In order to fit an SVM using a **non-linear kernel**, we once again use the `SVC()` function. However, now we use a different value of the parameter kernel. To fit an SVM with a polynomial kernel we use `kernel="poly"`, and to fit an SVM with a radial kernel we use `kernel="rbf"`. In the former case we also use the `degree` argument to specify a degree for the polynomial
kernel, and in the latter case we use `gamma` to specify a value of $\gamma$ for the radial basis kernel.

Let's generate some data with a non-linear class boundary:

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(8)
X = np.random.randn(200,2)
X[:100] = X[:100] +2
X[101:150] = X[101:150] -2
y = np.concatenate([np.repeat(-1, 150), np.repeat(1,50)])

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, random_state=2)

plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=mpl.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')

See how one class is kind of stuck in the middle of another class? This suggests that we might want to use a **radial kernel** in our SVM. Now let's fit the training data using the `SVC()` function with a radial kernel and $\gamma = 1$:

In [ ]:
svm = SVC(C=1.0, kernel='rbf', gamma=1)
svm.fit(X_train, y_train)
plot_svc(svm, X_test, y_test)

Not too shabby! The plot shows that the resulting SVM has a decidedly non-linear boundary. We can see from the figure that there are a fair number of training errors in this SVM fit. If we increase the value of cost, we can reduce the number of training errors:

In [ ]:
# Increasing C parameter, allowing more flexibility
svm2 = SVC(C=100, kernel='rbf', gamma=1.0)
svm2.fit(X_train, y_train)
plot_svc(svm2, X_test, y_test)

However, this comes at the price of a more irregular decision boundary that seems to be at risk of overfitting the data. We can perform cross-validation using `GridSearchCV()` to select the best choice of $\gamma$ and cost for an SVM with a radial kernel:

In [ ]:
tuned_parameters = [{'C': [0.01, 0.1, 1, 10, 100],
                     'gamma': [0.5, 1,2,3,4]}]
clf = GridSearchCV(SVC(kernel='rbf'), tuned_parameters, cv=10, scoring='accuracy')
clf.fit(X_train, y_train)
clf.best_params_

Therefore, the best choice of parameters involves `cost=1` and `gamma=0.5`. We can plot the resulting fit using the `plot_svc()` function, and view the test set predictions for this model by applying the `predict()` function to the test data:

In [ ]:
plot_svc(clf.best_estimator_, X_test, y_test)
print(confusion_matrix(y_test, clf.best_estimator_.predict(X_test)))
print(clf.best_estimator_.score(X_test, y_test))

87% of test observations are correctly classified by this SVM. Not bad!

## Comparing model performance

As we studied in the lecture and tested aboved, cross-validation is usually used to select the best set of hyperparameters (i.e. the ones that produces best performance). The common practice is to leave aside a test subset that is used to report the final evaluation of the model. This is precisely what we did above. Let's now extend this idea to the use of more than one algorithm. In a more real sceneario, it is typical (and recommended) to perform a model comparison analysis over as many ML algorithms as possible.

For instance, the following code estimate the cross-validated performance of a particular *svm* model:

In [ ]:
from sklearn.model_selection import cross_val_score

mdl = SVC(C=1, kernel='rbf')
scores = cross_val_score(mdl, X_train, y_train, cv=10)
print("Accuracy: mean %0.2f,  SE: %0.2f)" % (scores.mean(), scores.std()))

We may wish compare several models. Let's assume we would like to compare three SVM models which are implemented using three different kernel functions: `linear`, `poly` and `rbf`. In addition, we would like to verify that a logistic regression model doesn't do better than any of them. So, the strategy we will follow consists in comparing the estimated cross-validated performance of the models. In order to have a fairer comparison as possible, we should use the same data splits for all the estimations. To this, we must choose a cross-validator first. It could be `KFold()` which implements the standard CV. There are other options, please refer to the `sklearn` documentation.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

kcv = KFold(n_splits=10)

mdl_lr = LogisticRegression(max_iter = 10000)
mdl_svm1 = SVC(C=1, kernel='linear')
mdl_svm2 = SVC(C=1, kernel='poly', degree=5)
mdl_svm3 = SVC(C=1, kernel='rbf', gamma=50)

scores_lr = cross_val_score(mdl_lr, X_train, y_train, cv=kcv)
scores_svm1 = cross_val_score(mdl_svm1, X_train, y_train, cv=kcv)
scores_svm2 = cross_val_score(mdl_svm2, X_train, y_train, cv=kcv)
scores_svm3 = cross_val_score(mdl_svm3, X_train, y_train, cv=kcv)

print("LR Accuracy: mean %0.2f,  SE: %0.2f)" % (scores_lr.mean(), scores_lr.std()))
print("SVM-linear Accuracy: mean %0.2f,  SE: %0.2f)" % (scores_svm1.mean(), scores_svm1.std()))
print("SVM-poly Accuracy: mean %0.2f,  SE: %0.2f)" % (scores_svm2.mean(), scores_svm2.std()))
print("SVM-rbf Accuracy: mean %0.2f,  SE: %0.2f)" % (scores_svm3.mean(), scores_svm3.std()))


The issue with above approach is that we are not tuning SVM hyperparameters, so we surely are reporting suboptimal performances. 

The `LogisticRegression()` function includes `l1` and `l2` pensalisation terms (which represent *lasso* and *ridge*, as discussed in previous lectures). By default, the function implements `l2` with a hyperparameter `C`. This parameter works very similar to the SVM's *C* (smaller values specify stronger regularisation). The fillowing code searches for an optimal value for `C`:

In [ ]:
kcv = KFold(n_splits=10)

lr_hyp_grid = [{'C': [0.1, 1, 10, 100]}]
mdls_lr = GridSearchCV(LogisticRegression(), param_grid=lr_hyp_grid, cv=kcv, scoring='accuracy').fit(X_train, y_train)
mdls_lr.cv_results_

Before, we have just extracted the optimal hyperparameters. We use `cv_results_` to read all the search results.

Let's implement a similar code for SVMs:

In [ ]:
svm_hyp_grid = [
    {'C': [0.1, 1, 10, 100], 'kernel': ['linear']},
    {'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 0.5, 1, 100], 'kernel': ['rbf']}
]

mdls_svm = GridSearchCV(SVC(), param_grid=svm_hyp_grid, cv=kcv, scoring='accuracy').fit(X_train, y_train)

Now, let's put all the results together. We aim for creating an errorbar plot so we can form a better picture of the several model options we now have:

In [ ]:
scores_mean = np.concatenate((mdls_lr.cv_results_['mean_test_score'],
                             mdls_svm.cv_results_['mean_test_score']))

scores_std = np.concatenate((mdls_lr.cv_results_['std_test_score'],
                             mdls_svm.cv_results_['std_test_score']))

mdl_lbls = []
for pos in np.concatenate((mdls_lr.cv_results_['params'],mdls_svm.cv_results_['params'])):
    key_values = pos.items()
    mdl_lbls +=[', '.join([str(key)+ ':' + str(value) for key, value in key_values])]

plt.figure(figsize=(10,8))
plt.errorbar(scores_mean, mdl_lbls, xerr=scores_std, fmt='o', color='black',
             ecolor='lightgray', elinewidth=4, capsize=0)
plt.grid(True, )
plt.xlabel('Accuracy score')
plt.ylabel('Model')

Notice the difference in model performance across the model implementations. For this particular example, the worst models exhibit performances around 0.77, whilst the best ones are above 0.9. This shows the importance for model selection and validation.

Remember that we left aside a test data subset. We usually use it to report overall performances and final verifications. From the above plot we notice that two SVM-RBF models stand out: 1) $C=10$ and $\gamma=0.1$, and 2) $C=1$ and $\gamma=0.5$. We can use the test subset for a final decision:

In [ ]:
svm1 = SVC(C=10, kernel='rbf', gamma=0.1).fit(X_train, y_train)
svm2 = SVC(C=1, kernel='rbf', gamma=0.5).fit(X_train, y_train)

#y_pred1 = svm1.predict(X_test)
#y_pred2 = svm2.predict(X_test)

print('Test accuracy SVM1 = %0.2f' % (svm1.score(X_test, y_test)))
print('Test accuracy SVM2 = %0.2f' % (svm2.score(X_test, y_test)))

It seems the first SVM model is just slightly better than the second one.

## ROC Curves

The `auc()` function from the `sklearn.metrics` package can be used to produce ROC curves such as those we saw in previous lecture:

In [ ]:
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

We can use ROC analysis to get a finer picture about the two chosen models. In addition, let's include one of the LRs for illustration purpose:

In [ ]:
lr1 = LogisticRegression(C=0.1).fit(X_train, y_train)
print('Test accuracy LR = %0.2f' % (lr1.score(X_test, y_test)))

SVMs and support vector classifiers output class labels for each observation.
However, it is also possible to obtain fitted values for each observation,
which are the numerical scores used to obtain the class labels. For instance,
in the case of a support vector classifier, the fitted value for an observation
$X = (X_1,X_2, . . .,X_p)^T$ takes the form $\hat\beta_0 + \hat\beta_1X_1 + \hat\beta_2X_2 + . . . + \hat\beta_pX_p$.

For an SVM with a non-linear kernel, the equation that yields the fitted
value was given in the lecture slides. In essence, the sign of the fitted value determines
on which side of the decision boundary the observation lies. Therefore, the
relationship between the fitted value and the class prediction for a given
observation is simple: if the fitted value exceeds zero then the observation
is assigned to one class, and if it is less than zero than it is assigned to the
other.

In order to obtain the fitted values for a given SVM model fit, we
use the `.decision_function()` method of the SVC:

In [ ]:
y_train_score1 = svm1.decision_function(X_train)
y_train_score2 = svm2.decision_function(X_train)

Now we can produce the ROC plot to see how the models perform on both the training and the test data:

In [ ]:
false_pos_rate1, true_pos_rate1, _ = roc_curve(y_train, y_train_score1)
roc_auc1 = auc(false_pos_rate1, true_pos_rate1)

false_pos_rate2, true_pos_rate2, _ = roc_curve(y_train, y_train_score2)
roc_auc2 = auc(false_pos_rate2, true_pos_rate2)

y_train_probs = lr1.predict_proba(X_train)[:,1]
false_pos_rate3, true_pos_rate3, _ = roc_curve(y_train, y_train_probs)
roc_auc3 = auc(false_pos_rate3, true_pos_rate3)

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(14,6))
ax1.plot(false_pos_rate1, true_pos_rate1, label='SVM $C=10, \gamma = 0.1$ ROC curve (area = %0.2f)' % roc_auc1, color='b')
ax1.plot(false_pos_rate2, true_pos_rate2, label='SVM $C=1, \gamma = 0.5$ ROC curve (area = %0.2f)' % roc_auc2, color='r')
ax1.plot(false_pos_rate3, true_pos_rate3, label='LogReg $C=0.1$ ROC curve (area = %0.2f)' % roc_auc3, color='g')
ax1.set_title('Training Data')

y_test_score1 = svm1.decision_function(X_test)
y_test_score2 = svm2.decision_function(X_test)

false_pos_rate1, true_pos_rate1, _ = roc_curve(y_test, y_test_score1)
roc_auc1 = auc(false_pos_rate1, true_pos_rate1)

false_pos_rate2, true_pos_rate2, _ = roc_curve(y_test, y_test_score2)
roc_auc2 = auc(false_pos_rate2, true_pos_rate2)

y_test_probs = lr1.predict_proba(X_test)[:,1]
false_pos_rate3, true_pos_rate3, _ = roc_curve(y_test, y_test_probs)
roc_auc3 = auc(false_pos_rate3, true_pos_rate3)

ax2.plot(false_pos_rate1, true_pos_rate1, label='SVM $C=10, \gamma = 0.1$ ROC curve (area = %0.2f)' % roc_auc1, color='b')
ax2.plot(false_pos_rate2, true_pos_rate2, label='SVM $C=1, \gamma = 0.5$ ROC curve (area = %0.2f)' % roc_auc2, color='r')
ax2.plot(false_pos_rate3, true_pos_rate3, label='LogReg $C=0.1$ ROC curve (area = %0.2f)' % roc_auc3, color='g')
ax2.set_title('Test Data')

for ax in fig.axes:
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([-0.05, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc="lower right")

So, it is clearer now that the first SVM is better than the second one, as the gap between test AUCs is more significant now. It is interesnting to note the very poor performance of the LogReg model. **Why is this?**

## SVM with Multiple Classes

If the response is a factor containing more than two levels, then the `svm()`
function will perform multi-class classification using the one-versus-one approach.
We explore that setting here by generating a third class of observations:

In [ ]:
np.random.seed(8)
XX = np.vstack([X, np.random.randn(50,2)])
yy = np.hstack([y, np.repeat(0,50)])
XX[yy ==0] = XX[yy == 0] +4

plt.scatter(XX[:,0], XX[:,1], s=70, c=yy, cmap=plt.cm.prism)
plt.xlabel('XX1')
plt.ylabel('XX2')

Fitting an SVM to multiclass data uses identical syntax to fitting a simple two-class model:

In [ ]:
svm5 = SVC(C=1, kernel='rbf')
svm5.fit(XX, yy)
plot_svc(svm5, XX, yy)

## Application to Handwritten Letter Data

We now examine [Optical Recognition of Handwritten Digits Data Set](http://archive.ics.uci.edu/ml/datasets/optical+recognition+of+handwritten+digits), which contains 5,620 samples of handwritten digits 0..9. You can use these links to download the [training data](http://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra) and [test data](http://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes), and then we'll load them into python, or get them from the module GitHub's repository as in the example below:

In [ ]:
X_train = pd.read_csv('https://raw.githubusercontent.com/iaolier/7021DATSCI/main/data/optdigits.tra', header=None)
y_train = X_train[64]
X_train = X_train.drop(X_train.columns[64], axis=1)

X_test = pd.read_csv('https://raw.githubusercontent.com/iaolier/7021DATSCI/main/data/optdigits.tes', header=None)
y_test = X_test[64]
X_test = X_test.drop(X_test.columns[64], axis=1)

Let's take a look at the dimensions of this dataset:

In [ ]:
print(X_train.shape)
print(X_test.shape)

This data set consists of preprocessed images of handwriting samples gathered from 43 different people. Each image was converted into an 8x8 matrix (64 pixels), which was then flattened into a vector of 64 numeric values. The final column contains the class label for each digit.

The training and test sets consist of 3,823 and 1,797 observations respectively. Let's see what one of these digits looks like:

In [ ]:
plt.imshow(X_train.values[50].reshape(8,8), cmap="gray") 
plt.show()

That's a pretty messy digit. Let's peek at the true class:

In [ ]:
y_train[50]

It looks like our SVM has its work cut out for it! Let's start with a linear kernel to see how we do:

In [ ]:
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

# Print a nice confusion matrix
cm = confusion_matrix(y_train, svc.predict(X_train))
cm_df = pd.DataFrame(cm.T, index=svc.classes_, columns=svc.classes_)
print(cm_df)

We see that there are **no training errors**. In fact, this is not surprising,
because the large number of variables relative to the number of observations
implies that it is easy to find hyperplanes that fully separate the classes. We
are most interested not in the support vector classifier’s performance on the
training observations, but rather its performance on the test observations:

In [ ]:
cm = confusion_matrix(y_test, svc.predict(X_test))
print(pd.DataFrame(cm.T, index=svc.classes_, columns=svc.classes_))

## Exercise 1:
We see that using `cost = 10` yields just 71 test set errors on this data. Now try using the `GridSearchCV()` function to select an optimal value for `c`. Consider values in the range 0.01 to 100:

## Exercise 2: Support vector regression (SVR)
SVRs are implemented using the `sklearn` object `SVR` (details in: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). Use the *Hitters* dataset (https://raw.githubusercontent.com/iaolier/7021DATSCI/main/data/Hitters.csv) to build a model that predicts the salary as a function of the other variables. You should do your best to produce the best possible model among several SVR and multiple linear regression models. The final outcome of this task is an errorbar plot with all your models reported (similar to the one above for classification). 

In [ ]:
# Insert your code here.